# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods

# Submission for Week 1

### A description of the problem and a discussion of the background. 

We intend to open a new pizza place in Mumbai. There are many areas in Mumbai, classified on the basis of Pin Code. We intend to find the area which will be most sutaible to opning a Pizza Place. The area shold have relatively less competion and good amount of customers.

Details about Mumbai:Mumbai (Old name Bombay) is a major city in India. It is the most populous city in India with an estimated population of 12.4 million as of 2011. It is the financial, commercial and entertainment capital of India. https://en.wikipedia.org/wiki/Mumbai

### A description of the data and how it will be used to solve the problem.

Following data will be required for the research:
* List of Pin Codes in Mumbai: This data can be obtained from the website 'Maps of India'. url: https://www.mapsofindia.com/pincode/india/maharashtra/mumbai
* The geo coordinates (Latitude and Longitude) for these areas can be obtained from Google API. This data will help in plotting the areas on the map.
* From Foresquare API we will need information regarding 
    * Pizza places in Mumbai based on area
    * Schools/Colleges/Universities in Mumbai based on area
    * Offices in Mumbai based on area
* Form the foresquare dat we will have to extract
    * Venue
    * Venue Latitude
    * Venue Longitude
    * Venue Category

# Submission for Week 2

## Introduction

Mumbai (Old name Bombay) is a major city in India. It is the most populous city in India with an estimated population of 12.4 million as of 2011. It is the financial, commercial and entertainment capital of India. https://en.wikipedia.org/wiki/Mumbai

Objective:<br> We intend to open a new pizza place in Mumbai. There are many areas in Mumbai, classified on the basis of Pin Code. We intend to find the area which will be most sutaible to opning a Pizza Place. The area shold have relatively less competion and good amount of customers.

## Data

Following data will be required for the research:
* List of Pin Codes in Mumbai: This data can be obtained from the website 'Maps of India'. url: https://www.mapsofindia.com/pincode/india/maharashtra/mumbai
* The geo coordinates (Latitude and Longitude) for these areas can be obtained from Google API. This data will help in plotting the areas on the map.
* From Foresquare API we will need information regarding 
    * Pizza places in Mumbai based on area
    * Schools/Colleges/Universities in Mumbai based on area
    * Offices in Mumbai based on area
* Form the foresquare dat we will have to extract
    * Venue
    * Venue Latitude
    * Venue Longitude
    * Venue Category

## Methodology 

We will design a metric called 'Pizza shop metric'. Each locality will have a value corresponding to this metric. 

Steps:
* The number of each venue type in a particular area will be added. 
* This sum will be further multiplied with the multiplier
* The addition of all such venue types will be the metric for the area.

Multipliers:<br>
* -1 : Pizza places in Mumbai based on area <br>
* +1 : Schools/Colleges/Universities in Mumbai based on area<br>
* +2 : Offices in Mumbai based on area<br>



#### Import Libraries

In [4]:
#Inport required libraries
import pandas as pd
import numpy as np
import requests
from geopy import geocoders
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.96 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.99 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.61 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.02 MB/s
Libraries imported


#### Obtain data of Pincodes

In [5]:
#This website contains Pin codes of various areas in Mumbai
website_url = requests.get("https://finkode.com/mh/mumbai.html").text
soup = BeautifulSoup(website_url,'lxml')
My_table_mumbai = soup.find('table',{'class':'plist'})
A=[]
B=[]
C=[]
D=[]
#Load the data
for row in My_table_mumbai.findAll("tr"):
    cells = row.findAll("td")
    if cells !=[]:
        A.append(cells[0].find(text=True))
        B.append(cells[2].find(text=True))
df=pd.DataFrame(A,columns=['Area'])
df['Pin Code']=B
#df = df.iloc[1:]
#df = df.ix[df['Pin Code'] != 'Pincode ']
df.drop_duplicates(subset ='Pin Code', keep = 'first', inplace = True)
#df = df.groupby('Pin Code').Area.agg([('Area', ', '.join)])
print('Dataframe containg areas and Pin codes')
print(df.shape)
df.head()

Dataframe containg areas and Pin codes
(89, 2)


,Area,Pin Code
0,A I Staff Colony S.O,400029
1,Aareymilk Colony S.O,400065
2,Agripada S.O,400011
3,Ambewadi S.O (Mumbai),400004
4,Andheri East S.O,400069


#### Obtain coordinates of these Pin Codes

In [6]:
C=[]
D=[]
geolocator = Nominatim(user_agent="specify_your_app_name_here")
for index, row in df.iterrows():
    
    try:
        location = geolocator.geocode(row['Pin Code'])
        C.append(location.latitude)
        D.append(location.longitude)
    except AttributeError:
        C.append('No data found')
        D.append('No data found') 
    
df['Latitude']=C
df['Longitude']=D

#Remove 'No data found' fields
#df = df.ix[df['Latitude'] != 'No data found']

print('Dataframe containg areas, Pin codes and coordinates')
print(df.shape)
df.head()

Dataframe containg areas, Pin codes and coordinates
(89, 4)


,Area,Pin Code,Latitude,Longitude
0,A I Staff Colony S.O,400029,48.525070,44.586426
1,Aareymilk Colony S.O,400065,19.166890,72.854364
2,Agripada S.O,400011,18.983293,72.826819
3,Ambewadi S.O (Mumbai),400004,18.954329,72.821730
4,Andheri East S.O,400069,19.119432,72.851426


#### Plot the map of Mumbai

##### Plotting map of Mumbai centered around its coordinates

In [7]:
#Obtain the coordinates of Mumbai
address = 'Mumbai, India'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(df['Latitude'], df['Longitude'], df['Area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
#Display the map of Mumbai
print ('Map of Mumbai with areas mapped against Pin Codes')
map_mumbai

Map of Mumbai with areas mapped against Pin Codes


##### Function to add markers 

In [8]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, area, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Area'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, area)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

#### Get nearby venues using ForeSquare API

##### Function to get nearby venues

In [11]:
CLIENT_ID = 'XKCQNKBQZ5IIBB2SKC0OOS4SARS1NGA0I25GV4WT1WO4VRNK' # Foursquare ID
CLIENT_SECRET = 'BHYKLDW4ZDXZC5IB4VYX2KQEE0DSHC15VIBIXZXSO3LCI3LB' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, categoryID='', radius=1000):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        if (categoryID != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryID)
       
        print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)           

##### Lets obtain Pizza Places in the area

In [12]:
# 4bf58dd8d48988d1ca941735 - Category ID for Pizza Place
pizza_venues = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   categoryID='4bf58dd8d48988d1ca941735'
                                  )
print('Pizza venues containg areas, Pin codes and coordinates')
print(pizza_venues.shape)
print (pizza_venues['Venue Category'].unique())
pizza_venues.head()

https://api.foursquare.com/v2/venues/explore?&client_id=XKCQNKBQZ5IIBB2SKC0OOS4SARS1NGA0I25GV4WT1WO4VRNK&client_secret=BHYKLDW4ZDXZC5IB4VYX2KQEE0DSHC15VIBIXZXSO3LCI3LB&v=20180605&ll=48.5250698368735,44.5864264304804&radius=1000&limit=100&categoryId=4bf58dd8d48988d1ca941735


KeyError: 'groups'

In [3]:
# Plot pizza places on the map
map_mumbai_pizza_place = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(pizza_venues, 'red', map_mumbai)
print ('Map of Mumbai with Pizza Venues')
map_mumbai_pizza_place

NameError: name 'folium' is not defined

##### Lets obtain schools in the area

In [ ]:
# 4bf58dd8d48988d13b941735 - Category ID for Schools
academic_venues_school = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   categoryID='4bf58dd8d48988d13b941735'
                                  )
print('Academic venues - School containg areas, Pin codes and coordinates')
print(academic_venues_school.shape)
print (academic_venues_school['Venue Category'].unique())
academic_venues_school.head()

In [ ]:
# Plot schools on the map
map_mumbai_school = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(academic_venues_school, 'red', map_mumbai)
print ('Map of Mumbai with schools')
map_mumbai_school

##### Lets obtain colleges/universities in the area

In [ ]:
# 4d4b7105d754a06372d81259 - Category ID for College and University
academic_venues_college = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   categoryID='4d4b7105d754a06372d81259'
                                  )
print('Academic venues - College/University containg areas, Pin codes and coordinates')
print(academic_venues_college.shape)
print (academic_venues_college['Venue Category'].unique())
academic_venues_college.head()

In [ ]:
# Plot colleges/universities on the map
map_mumbai_college = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(academic_venues_college, 'red', map_mumbai)
print ('Map of Mumbai with colleges/universities')
map_mumbai_college

##### Lets obtain offices in the area

In [ ]:
# 4bf58dd8d48988d124941735 - Category ID for Offices
office_venues = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                   categoryID='4bf58dd8d48988d124941735'
                                  )
print('Office venues containg areas, Pin codes and coordinates')
print(office_venues.shape)
print (office_venues['Venue Category'].unique())
office_venues.head()

In [ ]:
# Plot offices on the map
map_mumbai_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(office_venues, 'red', map_mumbai)
print ('Map of Mumbai with offices')
map_mumbai_office

#### Prepaaring the metric for area selection

##### Function to find count of venues

In [ ]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Area').count()
    
    for n in startDf['Area']:
        try:
            startDf.loc[startDf['Area'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Area'] == n,columnTitle] = 0

##### Adding venue count to data frame

In [ ]:
df_venue_count = df.copy()
df_metric.rename(columns={'Area':'Area'}, inplace=True)
addColumn(df_venue_count, 'PizzaPlace', pizza_venues)
addColumn(df_venue_count, 'School', academic_venues_school)
addColumn(df_venue_count, 'College', academic_venues_college)
addColumn(df_venue_count, 'Office', office_venues)

print('Dataframe containg count of venues')
print(df_venue_count.shape)
df_venue_count.head()

##### Adding weighted average metric to dataframe

In [ ]:
#defining weights
weight_pizza = -1
weight_school = 1
weight_college = 1
weight_office = 2

df_metric = df_data[['Area']].copy()
df_metric['Metric'] = df_venue_count['PizzaPlace'] * weight_pizza + df_venue_count['School'] * weight_school + df_venue_count['College'] * weight_college + df_venue_count['Office'] * weight_office
df_metric = df_metric.sort_values(by=['Metric'], ascending=False)

print('Dataframe containg selection metric of venues')
print(df_metric.shape)
df_metric.head()

#### Ploting the metric on the  map